<a href="https://colab.research.google.com/github/onucharles/generative-models-cms/blob/master/Assignment3_part3_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth

auth.authenticate_user()

!pip install pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
# from google.colab import auth
from oauth2client.client import GoogleCredentials
#@title Functions
def upload_to_drive(filename):
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  to_upload = drive.CreateFile({'title': filename})
  to_upload.SetContentFile(filename)
  to_upload.Upload()
  return to_upload

In [2]:
!git clone --single-branch --branch master https://github.com/onucharles/generative-models-cms.git
!wait $!
!mv -v generative-models-cms/* .
!wait $!
!rm -r generative-models-cms
!wait $!

Cloning into 'generative-models-cms'...
remote: Enumerating objects: 153, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 153 (delta 90), reused 116 (delta 56), pack-reused 0
Receiving objects: 100% (153/153), 1.42 MiB | 508.00 KiB/s, done.
Resolving deltas: 100% (90/90), done.
renamed 'generative-models-cms/classify_svhn.py' -> './classify_svhn.py'
renamed 'generative-models-cms/density_estimation.py' -> './density_estimation.py'
renamed 'generative-models-cms/estimate_jsd_wd.py' -> './estimate_jsd_wd.py'
mv: cannot move 'generative-models-cms/figures' to './figures': Directory not empty
renamed 'generative-models-cms/models.py' -> './models.py'
renamed 'generative-models-cms/README.md' -> './README.md'
renamed 'generative-models-cms/samplers.py' -> './samplers.py'
renamed 'generative-models-cms/score_fid.py' -> './score_fid.py'
renamed 'generative-models-cms/svhn_classifier.pt' -> './svhn_classifier.pt'
renamed 'gen

In [0]:
import os
from torchvision.datasets import utils
import numpy as np
from models import VAE2, Elbo_Normal
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision.utils import save_image
import time
import os
import csv
import torchsummary
import io
from contextlib import redirect_stdout
from math import ceil
from util import load_svhn
import VAE_binary

**Set Hyperparameters**

In [0]:
#Hyperparameters
batch_size = 64
lr = 3e-4
epochs = 20
save_interval = 3 #save model every save_interval epochs (None for not saving)
log_interval = 100 #print results every log_interval batches (in addition to every epoch (None for not printing)
seed = 1111
np.random.seed(seed)
torch.random.manual_seed(seed)
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

**Load Data**

In [5]:
train_loader, val_loader, test_loader = load_svhn(os.curdir, batch_size=batch_size)

Using downloaded and verified file: ./train_32x32.mat
Using downloaded and verified file: ./test_32x32.mat


**Create Save Directory**

In [0]:
save_parent_dir = os.curdir
model_folder = "VAE_SVHN_model_"
previous_folders = [f for f in os.listdir(save_parent_dir) if (model_folder in f and os.path.isdir(f))]
if not previous_folders:
    save_dir = os.path.join(save_parent_dir, model_folder + f'{1:03d}')
else:
    last_model = max(previous_folders)
    i = int(last_model.replace(model_folder,"")) + 1
    save_dir = os.path.join(save_parent_dir, model_folder + f'{i:03d}')
os.mkdir(save_dir)

**Create Model**

In [0]:
model = VAE2()
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = Elbo_Normal()

In [8]:
train_samples, _ = next(iter(train_loader))
val_samples, _ = next(iter(val_loader))
random_z = torch.randn((batch_size, 100))
VAE_binary.generate_samples(model, save_dir=save_dir, epoch=0, train_samples=train_samples, val_samples=val_samples, random_z=random_z, model_outputs_logits=False)
VAE_binary.print_model_summary(model, optimizer, save_dir=save_dir, input_size=(3,32,32))

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0003
    weight_decay: 0
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             896
               ELU-2           [-1, 32, 32, 32]               0
         AvgPool2d-3           [-1, 32, 16, 16]               0
            Conv2d-4           [-1, 64, 16, 16]          18,496
               ELU-5           [-1, 64, 16, 16]               0
         AvgPool2d-6             [-1, 64, 8, 8]               0
            Conv2d-7            [-1, 256, 8, 8]         147,712
               ELU-8            [-1, 256, 8, 8]               0
         AvgPool2d-9            [-1, 256, 4, 4]               0
           Conv2d-10            [-1, 256, 1, 1]       1,048,832
              ELU-11            [-1, 256, 1, 1]               0
           Linear-12                  [-1, 100]

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2423: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


**Train Model**

In [9]:
train_elbos, val_elbos, epoch_time = VAE_binary.train(model, optimizer, train_loader, val_loader, loss_fn, epochs=epochs,
                                           save_dir=save_dir, save_interval=save_interval, log_interval=log_interval, 
                                           model_outputs_logits=False, train_samples=train_samples, val_samples=val_samples, random_z=random_z)

----> Epoch 0,	 Batch 0,	 Train ELBO: -3120.98


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2423: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


----> Epoch 0,	 Batch 100,	 Train ELBO: -3000.89
----> Epoch 0,	 Batch 200,	 Train ELBO: -2979.97
----> Epoch 0,	 Batch 300,	 Train ELBO: -2968.22
----> Epoch 0,	 Batch 400,	 Train ELBO: -2960.12
----> Epoch 0,	 Batch 500,	 Train ELBO: -2954.35
----> Epoch 0,	 Batch 600,	 Train ELBO: -2949.89
----> Epoch 0,	 Batch 700,	 Train ELBO: -2946.20
----> Epoch 0,	 Batch 800,	 Train ELBO: -2943.20
----> Epoch 0,	 Batch 900,	 Train ELBO: -2940.62
----> Epoch 0,	 Batch 1000,	 Train ELBO: -2938.54
-> Epoch 0,	 Train ELBO: -2937.89,	 Validation ELBO: -2917.26,	 Max Validation ELBO: -2917.26,	 Epoch Time: 21.79 seconds
----> Epoch 1,	 Batch 0,	 Train ELBO: -2905.78
----> Epoch 1,	 Batch 100,	 Train ELBO: -2916.26
----> Epoch 1,	 Batch 200,	 Train ELBO: -2916.36
----> Epoch 1,	 Batch 300,	 Train ELBO: -2916.28
----> Epoch 1,	 Batch 400,	 Train ELBO: -2915.85
----> Epoch 1,	 Batch 500,	 Train ELBO: -2915.70
----> Epoch 1,	 Batch 600,	 Train ELBO: -2915.49
----> Epoch 1,	 Batch 700,	 Train ELBO: -2915.

In [12]:
!zip -r VAE_SVHN_model_001.zip VAE_SVHN_model_001
upload_to_drive('VAE_SVHN_model_001.zip')

  adding: VAE_SVHN_model_012/ (stored 0%)
  adding: VAE_SVHN_model_012/generated_val_samples_best.png (deflated 0%)
  adding: VAE_SVHN_model_012/generated_train_samples_best.png (deflated 0%)
  adding: VAE_SVHN_model_012/generated_random_samples_3.png (deflated 0%)
  adding: VAE_SVHN_model_012/original_train_samples.png (deflated 0%)
  adding: VAE_SVHN_model_012/generated_val_samples_9.png (deflated 0%)
  adding: VAE_SVHN_model_012/generated_random_samples_best.png (deflated 0%)
  adding: VAE_SVHN_model_012/generated_random_samples_9.png (deflated 0%)
  adding: VAE_SVHN_model_012/stats.csv (deflated 47%)
  adding: VAE_SVHN_model_012/generated_val_samples_6.png (deflated 0%)
  adding: VAE_SVHN_model_012/model_epoch_0.pt (deflated 9%)
  adding: VAE_SVHN_model_012/generated_train_samples_6.png (deflated 0%)
  adding: VAE_SVHN_model_012/generated_random_samples_6.png (deflated 0%)
  adding: VAE_SVHN_model_012/model_epoch_9.pt (deflated 8%)
  adding: VAE_SVHN_model_012/model_epoch_6.pt (def

GoogleDriveFile({'title': 'VAE_SVHN_model_012.zip', 'mimeType': 'application/zip', 'kind': 'drive#file', 'id': '1nPKFBweeU2hec-1obWg0G5OhjzSAxwEw', 'etag': '"O-ZzQh-gY65a42kIgC9WL9_4IwE/MTU1NTU2NjA3NTY1OQ"', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1nPKFBweeU2hec-1obWg0G5OhjzSAxwEw', 'webContentLink': 'https://drive.google.com/uc?id=1nPKFBweeU2hec-1obWg0G5OhjzSAxwEw&export=download', 'alternateLink': 'https://drive.google.com/file/d/1nPKFBweeU2hec-1obWg0G5OhjzSAxwEw/view?usp=drivesdk', 'embedLink': 'https://drive.google.com/file/d/1nPKFBweeU2hec-1obWg0G5OhjzSAxwEw/preview?usp=drivesdk', 'iconLink': 'https://drive-thirdparty.googleusercontent.com/16/type/application/zip', 'labels': {'starred': False, 'hidden': False, 'trashed': False, 'restricted': False, 'viewed': True}, 'copyRequiresWriterPermission': False, 'createdDate': '2019-04-18T05:41:15.659Z', 'modifiedDate': '2019-04-18T05:41:15.659Z', 'modifiedByMeDate': '2019-04-18T05:41:15.659Z', 'lastViewedByMeDate': '2019-04